<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import (
    layers,
    models,
    callbacks,
    losses,
    utils,
    metrics,
    optimizers,
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Prepare the data

In [3]:
!pip install kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"ahtesham","key":"cee6ab1f831fe575d0f17c5bf9ea218b"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!kaggle datasets download -d joosthazelzet/lego-brick-images -p /content


 99% 0.98G/1.00G [00:09<00:00, 184MB/s]
100% 1.00G/1.00G [00:10<00:00, 107MB/s]


In [8]:
!unzip /content/lego-brick-images.zip -d /content/lego-brick-images


Streaming output truncated to the last 5000 lines.
  inflating: /content/lego-brick-images/dataset/43857 beam 1x2 300R.png  
  inflating: /content/lego-brick-images/dataset/43857 beam 1x2 301L.png  
  inflating: /content/lego-brick-images/dataset/43857 beam 1x2 301R.png  
  inflating: /content/lego-brick-images/dataset/43857 beam 1x2 302L.png  
  inflating: /content/lego-brick-images/dataset/43857 beam 1x2 302R.png  
  inflating: /content/lego-brick-images/dataset/43857 beam 1x2 303L.png  
  inflating: /content/lego-brick-images/dataset/43857 beam 1x2 303R.png  
  inflating: /content/lego-brick-images/dataset/43857 beam 1x2 304L.png  
  inflating: /content/lego-brick-images/dataset/43857 beam 1x2 304R.png  
  inflating: /content/lego-brick-images/dataset/43857 beam 1x2 305L.png  
  inflating: /content/lego-brick-images/dataset/43857 beam 1x2 305R.png  
  inflating: /content/lego-brick-images/dataset/43857 beam 1x2 306L.png  
  inflating: /content/lego-brick-images/dataset/43857 beam 1x

#0.Parameters

In [9]:
IMAGE_SIZE  = 64
CHANNELS = 1
BATCH_SIZE =128
Z_DIM = 100
EPOCHS = 300
LOAD_MODEL = False
ADAM_BETA_1 = 0.5
ADAM_BETA_2 = 0.999
LEARNING_RATE = 0.0002
NOISE_PARAM = 0.1


#1.Prepare the data

In [10]:
train_data = utils.image_dataset_from_directory(
    "/content/lego-brick-images/",
    labels = None,
    color_mode = "grayscale",
    image_size = (IMAGE_SIZE , IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = True,
    seed = 42,
    interpolation = "bilibear",
)

NotImplementedError: ignored

In [11]:
def preprocess(img):
  """normalize and reshape the images"""

  img = (tf.cast(img , "float32") - 127.5)/127.5
  return img


train = train_data.map(lambda x :preprocess(x))


NameError: ignored

In [12]:
train_sample = sample_batch(train)

NameError: ignored

In [13]:
display(train_sample)

NameError: ignored

#2.Build the GAN

In [15]:
discriminator_input = layers.Input(shape = (IMAGE_SIZE , IMAGE_SIZE , CHANNELS))
x = layers.Conv2D(64, kernel_size = 4, strides = 2, padding = "same" , use_bias = False)(
    discriminator_input
)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    128, kernel_size = 4, strides = 2, padding = "same" , use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9) (x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    256, kernel_size = 4 , strides = 2, padding = "same", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    512 , kernel_size = 4, strides =2, padding = "same" , use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    1,
    kernel_size = 4,
    strides = 1,
    padding = "valid",
    use_bias = False,
    activation = "sigmoid",
)(x)
discriminator_output = layers.Flatten()(x)

discriminator = models.Model(discriminator_input, discriminator_output)
discriminator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 64)        1024      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                                 
 dropout_2 (Dropout)         (None, 32, 32, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 128)       131072    
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 128)      512       
 hNormalization)                                                 
                                                             

In [1]:
#Genrator
generator_input = layers.Input(shape=(Z_DIM,))
x = layers.Rshape((1,1,Z_DIM))(genrator_input)
x = layers.Conv2DTranspose(
    512, kernel_size= 4, strides = 1 , padding = "valid", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Conv2DTranspose(
    256 , kernel_size = 4 , strides = 2, padding = "same", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Conv2DTranspose(
    128, kernel_size = 4, strides = 2, padding = "same", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Conv2DTranspose(
    64, kernel_size = 4, strides = 2, padding = "same", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
generator_output = layers.Conv2DTranspose(
    CHANNELS,
    kernel_size = 4,
    strides = 2,
    padding = "same",
    use_bias = False,
    activation = "tanh",
)(x)
genrator = models.Model(generator_input, genreator_output)
genrator.summary()

NameError: ignored

In [2]:
from tensorflow.python.ops.gradients_impl import gradients
class DCGAN(models.Model):
  def __init__(self, discriminator, genrator , latent_dim):
    super(DCGAN, self).__init__()
    self.generator = generator
    self.latent_dim = latent_dim

  def compile(self, d_optimizer, g_optimizer):
    super(DCGAN, self).compile()
    self.loss_fn = losses.BinaryCrossentropy()
    self.d_optimizer = d_optimizer
    self.g_optimizer = g_optimizer
    self.d_loss_metrics = metrics.Mean(name = "d_loss")
    self.d_real_acc_metric = metrics.BinaryAccuracy(name = "d_real_acc")
    self.d_fake_acc_metric = metrics.BinaryAccuracy(anme = "d_fake_acc")
    self.d_acc_metric = metrics.BinaryAccuracy(name = "d_acc")
    self.loss_metric = metrics.Mean(name = "g_loss")
    self.g_acc_metric = metrics.BinaryAccuracy(name = "g_acc")


  @property
  def metrics(self):
    return[
        self.d_loss_metric,
        self.d_real_acc_metric,
        self.d_fake_acc_metric,
        self.d_acc_metric,
        self.g_loss_metric,
        self.g_acc_metric,
    ]

  def train_step(self, real_images):
    #Sample random points in the latent space
    batch_size = tf.shape(real_images)[0]
    random_latent_vectors = tf.random.normal(
        shape = (batch_size, self.latent_dim)
    )

    #Train the discriminator on the fakes images
    with tf.GradientTape() as gen_tape, tf.GradientTape() as dics_tape:
      generated_images = self.generator(
          random_latent_vectors, training = True
      )
      real_predictions = self.discriminator(real_images,training = True)
      fake_predictions = self.discriminator(
          generated_images, training =True
      )

      real_labels = tf.ones_like(real_predictions)
      real_noisy_labels = real_labels +NOISE_PARAM * tf.random.uniform(
          tf.shape(real_predictions)
      )
      fake_labels = tf.zeros_like(fake_predictions)
      fake_noisy_labels = fake_labels - NOISE_PARAM * tf.random.uniform(
          tf.shape(fake_predictions)
      )

      d_real_loss = self.loss_fn(real_noisy_labels, real_predictions)
      d_fake_loss = self.loss_fn(fake_noisy_labels, fake_predictions)
      d_loss = (d_real_loss + d_fake_loss)/ 2.0

      g_loss = self.loss_fn(real_labels, fake_predictions)

      gradients_of_discriminator = dics_tape.gradient(
          d_loss, self.discriminator.traininable_variables
      )
      gradients_of_generator = gen_tape.gradient(
          g_loss, self.genrator.trainable_variables
      )

      self.d_optimizer.apply_gradients(
          zip(gradients_of_discriminator, discriminator.trainable_variables)
      )
      self.g_optimizer.apply_gradients(
          zip(gradients_of_generator, genrator.trainable_variables)
      )


      #update Metrics
      self.d_loss_metric.update_state(d_loss)
      self.d_real_acc_metric.update_state(real_labels, real_predictions)
      self.d_fake_Acc_metric.update_State(fake_labels, fake_predictions)
      self.d_acc_metric.update_state(
          [real_labels, fake_labels] , [real_predictions , fake_predictions]

      )
      self.g_loss_metric.update_state(g_loss)
      self.g_acc_metric.update_state(real_labels, fake_predictions)

      return {m.name : m.result() for m in self.metrics}


NameError: ignored

In [3]:
#Create a DCGAN
dcgan = DCGAN(
    discriminator = discriminator, genrator = genrator, latent_dim = Z_DIM
)

NameError: ignored

In [4]:
if LOAD_MODEL:
  dcgan.load_weights("./checkpoint/checkpoint.ckpt")

NameError: ignored

#3. Train the GAN

In [5]:
dcgan.compile(
    d_optimizer = optimizers.Adam(
        learning_rate = LEARNING_RATE, beta_1 = ADAM_BETA_1, beta_2 = ADAM_BETA_2
    ),
    g_optimizer = optimizers.Adam(
        learning_rate = LEARNING_RATE, beta_1 = ADAM_BETA_1 , beta_2 = ADAM_BETA_2
    ),
)

NameError: ignored

In [7]:
#Create a model save checkpoint

model_chekpoint_callback = callbacks.ModelCheckpoint(
    filepath = "./checkpoint/checkpoint.ckpt",
    save_weights_only = True,
    save_freq = "epoch",
    verbose = 0,
    )

tensorboard_callback = callbacks.TensorBoard(log_dir = "./logs")

class ImageGenerator(callbacks.Callbacks):
  def __init__(self, num_img, latent_dim):
    self.num_img = num_img
    self.latent_dim = latent_dim

  def on_epoch_end(self, epoch, logs = None):
    random_latent_vectors = tf.random.normal(
        shape = (self.num_img , self.latent_dim)
    )
    generated_images =self.model.generator(random_latent_vectors)
    generated_images = generated_images * 127.5 +127.5
    generated_images = generated_images.numpy()
    display(
        generated_images,
        save_to ="./output/genrated_img_%03d.png" % (epoch),
    )


NameError: ignored

In [8]:
#Save the final mdels
generator.save("./models/generator")
discriminator.save("./models/discriminator")

NameError: ignored

#3.Generate new images

In [9]:
#sample some points in the latent space, from the standard normal distribution
grid_width , grid_height = (10, 3)
z_sample = np.random.normal(size = (grid_width * grid_height, Z_DIM))

NameError: ignored

In [10]:
#Decode the sampled points
reconstructions = generator.predict(z_sample)

NameError: ignored

In [11]:
#Draw a plot of decoded images
fig = plt.figure(figsize = (18, 5))
fig.subplots_adjust(hspace = 0.4, wspace = 0.4)

#output the grid of faces
for i in range(grid_width * grid_height):
  ax = fig.add_subplot(grid_height, grid_weight , i+1)
  ax.axis("off")
  ax.imshow(reconstructions[i, :, :] , cmap = "Greys")


NameError: ignored

In [12]:
def compare_images(img1, img2):
  return np.mean(np.abs(img1 - img2))

In [13]:
all_data= []
for i in train.as_numpy_iterator():
  all_data.extend(i)
all_data = np.array(all_data)

NameError: ignored

In [14]:
r,c = 3,5
fig, axs = plt.subplots(r,c,figsize = (10, 6))
fig.subtitle("Genrated Images" , fontsize = 20)

noise = np.random.normal(size = (r * c , Z_DIM))
gen_imgs = genrator.predict(noise)

cnt = 0
for i in range(r):
  for j in range(c):
    axs[i,j].imshow(gen_imgs[cnt], cmap="gray_r")
    axs[i,j].axis("off")
    cnt += 1

plt.show()





NameError: ignored

In [15]:
fig, axs = plt.subplots(r, c, figsize = (10, 6))
fig.subtitle("Closest images in the training set" , fontsize = 20)

cnt = 0
for i in range(r):
  for j in range(c):
    c_diff = 99999
    c_img = None
    for k_idx, k in enumerate(all_data):
      diff = compare_images(gen_imgs[cnt], k)
      if diff < c_diff:
        c_img = np.copy(k)
        c_diff = diff

    axs[i, j].imshow(c_img , cmap="gray_r")
    axs[i , j].axis("off")
    cnt += 1

plt.show()

NameError: ignored